In [74]:
#ERMBOT Collecting data and Creating a ML model to get similar question and print their answer

In [75]:
#Finding the closest question and then anwering it

In [76]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [77]:
df1= pd.read_excel("Book1.xlsx")
df2= pd.read_excel("Book2.xlsx")

In [78]:
df1.head(20)


,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,data/set4/a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,data/set4/a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,data/set4/a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,data/set4/a10
5,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Yes,easy,easy,data/set4/a10
6,Alessandro_Volta,Who did Alessandro Volta marry?,Alessandro Volta married Teresa Peregrini.,medium,medium,data/set4/a10
7,Alessandro_Volta,Who did Alessandro Volta marry?,Teresa Peregrini,medium,medium,data/set4/a10
8,Alessandro_Volta,What did Alessandro Volta invent in 1800?,"In 1800, Alessandro Volta invented the voltaic...",medium,easy,data/set4/a10
9,Alessandro_Volta,What did Alessandro Volta invent in 1800?,voltaic pile,medium,medium,data/set4/a10


In [79]:
all_data = df1.append(df2)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3173 entries, 0 to 1714
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              3173 non-null   object
 1   Question                  3136 non-null   object
 2   Answer                    2697 non-null   object
 3   DifficultyFromQuestioner  2486 non-null   object
 4   DifficultyFromAnswerer    2695 non-null   object
 5   ArticleFile               3171 non-null   object
dtypes: object(6)
memory usage: 173.5+ KB


In [80]:
all_data.isnull().sum()

ArticleTitle                  0
Question                     37
Answer                      476
DifficultyFromQuestioner    687
DifficultyFromAnswerer      478
ArticleFile                   2
dtype: int64

In [81]:
all_data.dropna(inplace=True)

In [82]:
all_data['Question'] = all_data['ArticleTitle'].str.replace('_', ' ') + ' ' + all_data['Question']
all_data = all_data[['Question', 'Answer']]
all_data.shape

(2176, 2)

In [83]:
all_data

,Question,Answer
0,Alessandro Volta Was Alessandro Volta a profes...,Alessandro Volta was not a professor of chemis...
1,Alessandro Volta Was Alessandro Volta a profes...,No
2,Alessandro Volta Did Alessandro Volta invent t...,Alessandro Volta did invent the remotely opera...
3,Alessandro Volta Did Alessandro Volta invent t...,Yes
4,Alessandro Volta Was Alessandro Volta taught i...,Volta was taught in public schools.
...,...,...
1698,Woodrow Wilson Did Wilson support desegregation?,no
1699,Woodrow Wilson Did Wilson support desegregation?,Not in the slightest.
1700,Woodrow Wilson Did Wilson support the committe...,no
1701,Woodrow Wilson Did Wilson support the committe...,No.


In [84]:
all_data = all_data.drop_duplicates(subset='Question')
all_data.shape

(1198, 2)

In [85]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [86]:
stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()


def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [87]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(all_data['Question']))
print(tfidf_matrix.shape)


(1198, 1990)


In [88]:
def ask_question(question):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    max_similarity = np.argmax(similarity, axis=None)
    
    print('Your question:', question)
    print('Closest question found:', all_data.iloc[max_similarity]['Question'])
    print('Similarity: {:.2%}'.format(similarity[0, max_similarity]))
    print('Answer:', all_data.iloc[max_similarity]['Answer'])

In [89]:
ask_question('When Abraham Lincoln started his political career')

Your question: When Abraham Lincoln started his political career
Closest question found: Abraham Lincoln Did Lincoln start his political career in 1832?
Similarity: 86.68%
Answer: Yes


In [96]:
ask_question('elephant are big')

Your question: elephant are big
Closest question found: elephant Are elephants mammals?
Similarity: 49.71%
Answer: yes


In [ ]:
#GCP

In [ ]:
all_data.to_gbq(destination_table = 'Gmail_emails.email_data',project_id = '',if_exists = 'replace')
print('email data successfully updated to BigQueqy.')